In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
dirname=os.getcwd()
file_list=os.listdir(dirname+'\\input_data')

In [3]:
for file in file_list:
    if '权重' in file:
        file_list.remove(file)

In [4]:
all=[]
for i in file_list:
    temp=pd.read_excel('input_data\\'+i,sheet_name='规格品汇总')
    temp=temp.iloc[3:,0:3]
    temp = temp.rename(columns={'Unnamed: 2':i[:-5], 'Unnamed: 1':'规格'})
    temp.set_index(['表2. 各规格品汇总表','规格'],inplace=True)
    all.append(temp)
new_file=pd.concat(objs=all,axis=1)
new_file.reset_index(inplace=True)

In [5]:
new_file['表2. 各规格品汇总表']=new_file.iloc[:,0].str[:7]
df_final=new_file[new_file!=0]

In [6]:
test=df_final.drop(['表2. 各规格品汇总表','规格'],axis=1)
nanchang=df_final['南昌']
def cal_opposite(x):
    return x/nanchang
test=test.apply(cal_opposite)

#拼接表格
df_last=df_final[['表2. 各规格品汇总表','规格']].join(test)

In [7]:
#除去规格，按照各规格汇总进行分组
group=df_last.drop('规格',axis=1)
grouped=group.groupby(['表2. 各规格品汇总表'])

In [8]:
#使用函数进行求几何平均数
def geo_mean(iterable):
    return np.prod(iterable)**(1.0/len(iterable))
result=grouped.agg(geo_mean)
result.to_excel('GEKS.xlsx',encoding='utf8')